## For all $\rho$ values and all steps 10000 runs $\alpha=0$

## k_1=0 and  k_2=0.001 (varied for different transaction costs on each of the assets)

In [ ]:
reset()# Clears all the parameters
import numpy as np
import pandas as pd
from scipy.stats import norm
import math
from sage.misc.table import table
import matplotlib.pyplot as plt
from scipy.integrate import quad


def black_scholes_calculator(S, K, r, T, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    black_scholes_call_price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    return black_scholes_call_price

def delta_calculator(S, K, r, tau, sigma):
    if tau > 0:
        d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        return norm.cdf(d1)
    else:
        return 1 if S >= K else 0

def stock_price_simulator(S0_1, S0_2, mu_1, mu_2, sigma_1, sigma_2, T, N, rho):
    
    dt = T / N
    stock_path_1 = [S0_1]
    stock_path_2 = [S0_2]
    
    for _ in range(1, N +1): 
        Z1 = np.random.normal()
        Z2 = rho * Z1 + np.sqrt(1 - rho**2) * np.random.normal()

        S1_new = stock_path_1[-1] * np.exp((mu_1 - 0.5 * sigma_1**2) * dt + sigma_1 * np.sqrt(dt) * Z1)
        S2_new = stock_path_2[-1] * np.exp((mu_2 - 0.5 * sigma_2**2) * dt + sigma_2 * np.sqrt(dt) * Z2)

        stock_path_1.append(S1_new)
        stock_path_2.append(S2_new)

    return np.array(stock_path_1), np.array(stock_path_2)

def Unhedged_value(St, K, mu, sigma, T):
    zt = -1/(sigma* sqrt(T))*((np.log(St/K) + (mu - 0.5 * sigma**2) * T))
    
    def integrand_St(zt):
        part1 = np.exp((mu - 0.5 * sigma**2) * T + sigma*sqrt(T)*zt)
        part2 = np.exp(-0.5*(zt**2)) 
        return (part1 * part2) / np.sqrt(2 * np.pi )
    
    def integrand_K(zt):
        return np.exp(-0.5*(zt**2)) / np.sqrt(2 * np.pi)
    
    integral_St, _ = quad(integrand_St, zt, np.inf)
    integral_K, _ = quad(integrand_K, zt, np.inf)
    
    C = np.exp(-r * T) * (St*integral_St - K * integral_K)
    
    return C

def rebalancing_days(N,num_steps):
    return [round(i*N//num_steps) for i in range(1,num_steps+1)]

def portfolio_calculator(S0_1, S0_2, K, r, T, sigma_1, sigma_2, num_steps,N, mu_1, mu_2, rho, alpha):
    stock_path_1, stock_path_2 = stock_price_simulator(S0_1, S0_2, mu_1, mu_2, sigma_1, sigma_2, T, N, rho)
    sigma=sqrt((alpha**2)*(sigma_1**2) +  ((1-alpha)**2)*(sigma_2**2)  +  2 * rho*(alpha)*(1-alpha) * sigma_1 * sigma_2)
    dt = T/N
    rebalancing_day=rebalancing_days(N,num_steps)
        
    initial_portfolio_price_St = (alpha*S0_1+(1-alpha)*S0_2)
    
    black_scholes_callprice = black_scholes_calculator(initial_portfolio_price_St, K, r, T, sigma)
    delta_initial_1 = delta_calculator(S0_1, K, r, T, sigma_1)
    delta_initial_2 = delta_calculator(S0_2, K, r, T, sigma_2)

    portfolio_value_1_0 = S0_1 * delta_initial_1  * (1-k)
    portfolio_value_1_1 = S0_1 * delta_initial_1  * (1-k_1)

    portfolio_value_2_0 = S0_2 * delta_initial_2  * (1-k)
    portfolio_value_2_2 = S0_2 * delta_initial_2  * (1-k_2)

    
    tau=1

    """
    # Header for the table
    print(f"{'Period':.6}\t{'tau':.10}\t{'delta':<10}\t{'delta_before':.15}\t\t{'St2':.10}\t\t{'St':.10}\t{'Portfolio Value':.15}")
    print(f"{0:<6}\t{tau:<10}{round(delta_initial_1, 4):<10.4f}\t{'':<10}{round(S0_2, 2):<10.2f}\t{round(initial_portfolio_price_St, 2):<10.2f}\t{round(portfolio_value_2_2, 2):<20.2f}")
    """
    
    delta_before_1 = delta_initial_1
    delta_before_2 = delta_initial_2
    for t in range(1,N+1):

        St_1 = stock_path_1[t]
        St_2 = stock_path_2[t]
        tau = T - (t * dt)  

        if t in rebalancing_day:
            if tau >0:
                current_portfolio_price_St_1 = St_1 
                current_portfolio_price_St_2 = St_2 

                delta_1 = delta_calculator(current_portfolio_price_St_1, K, r, tau, sigma_1)
                delta_2 = delta_calculator(current_portfolio_price_St_2, K, r, tau, sigma_2)

                portfolio_value_1_0 = (1 + r * dt) * portfolio_value_1_0 + (delta_1 - delta_before_1) * current_portfolio_price_St_1 * (1 - np.sign(delta_1 - delta_before_1) * k)
                portfolio_value_1_1 = (1 + r * dt) * portfolio_value_1_1 + (delta_1 - delta_before_1) * current_portfolio_price_St_1 * (1 - np.sign(delta_1 - delta_before_1) * k_1)
                portfolio_value_2_0 = (1 + r * dt) * portfolio_value_2_0 + (delta_2 - delta_before_2) * current_portfolio_price_St_2 * (1 - np.sign(delta_2 - delta_before_2) * k)
                portfolio_value_2_2 = (1 + r * dt) * portfolio_value_2_2 + (delta_2 - delta_before_2) * current_portfolio_price_St_2 * (1 - np.sign(delta_2 - delta_before_2) * k_2)
                """
                print(f"{t:<6}\t{round(tau, 2):<10.2f}{round(delta_2, 4):<10.4f}\t{round(delta_before_2, 4):<10.4f}{round(St_2, 2):<10.2f}\t{round(current_portfolio_price_St_2, 2):<10.2f}\t{round(portfolio_value_2_2, 2):<20.2f}")
                """
                delta_before_1 = delta_1
                delta_before_2 = delta_2

            if tau==0:

                St_1_final = St_1   
                St_2_final = St_2   
                S_T = (alpha*St_1_final+(1-alpha)*St_2_final)

                delta_final_1 = delta_calculator(St_1_final, K, r, 0, sigma_1)
                delta_final_2 = delta_calculator(St_2_final, K, r, 0, sigma_2)

                final_portfolio_value_1_0 =(1 + r * dt) * portfolio_value_1_0 + (delta_final_1 - delta_before_1) * St_1_final * (1 - np.sign(delta_final_1 - delta_before_1) * k)- (delta_final_1 * St_1_final) + max(S_T - K, 0)
                simulatedprice_1_0 = (np.exp(-r * T)) * final_portfolio_value_1_0
                final_portfolio_value_1_1=(1 + r * dt) * portfolio_value_1_1 + (delta_final_1 - delta_before_1) * St_1_final * (1 - np.sign(delta_final_1 - delta_before_1) * k_1)- (delta_final_1 * St_1_final) + max(S_T - K, 0)
                simulatedprice_1_1 = (np.exp(-r * T)) * final_portfolio_value_1_1
                final_portfolio_value_2_0 =(1 + r * dt) * portfolio_value_2_0 + (delta_final_2 - delta_before_2) * St_2_final * (1 - np.sign(delta_final_2 - delta_before_2) * k)- (delta_final_2 * St_2_final) + max(S_T - K, 0)
                simulatedprice_2_0 = (np.exp(-r * T)) * final_portfolio_value_2_0

                final_portfolio_value_2_2 =(1 + r * dt) * portfolio_value_2_2 + (delta_final_2 - delta_before_2) * St_2_final * (1 - np.sign(delta_final_2 - delta_before_2) * k_2)- (delta_final_2 * St_2_final) + max(S_T - K, 0)
                simulatedprice_2_2 = (np.exp(-r * T)) * final_portfolio_value_2_2

                C_T = max(S_T - K, 0) 

                Measure = float((np.exp(-r * T)) * C_T)
                mu = (alpha*mu_1+(1-alpha)*mu_2)
                unhedged_value = Unhedged_value(initial_portfolio_price_St, K, mu, sigma,T)  

                delta_before_1 = delta_1
                delta_before_2 = delta_2

                return simulatedprice_1_0,simulatedprice_1_1,simulatedprice_2_0,simulatedprice_2_2,S_T,Measure,unhedged_value


    
S0_1 = 50     
S0_2 = 50         
#initial_portfolio_price_St=(alpha*S0_1+(1-alpha)*S0_2)
K = 50        
r = 0.02        
T = 1             
sigma_1 = 0.2      
sigma_2 = 0.2 
#sigma=sqrt((alpha**2)*(sigma_1**2) +  ((1-alpha)**2)*(sigma_2**2)  +  2 * rho*(alpha)*(1-alpha) * sigma_1 * sigma_2)

mu_1 = 0.05    
mu_2 = 0.05     
k=0.0 # zero transaction costs
N = 252 

num_simulations = 10000
alpha_values = np.round(np.linspace(0, 0, 1), 1)  # alpha=0
num_steps_list = [2,4, 12,52, 252]
rho_values = [0.2,0.3,0.35,0.5,0.55,0.7,0.8,0.99]
k_2_values = [0.001] # transaction costs on the second asset
k_1_values = [0.00] # transaction costs on the first asset

hedging_the_right_and_wrong_asset_alpha_1_SECTION_3_2_2=[]
for alpha in alpha_values:
    for k_1 in k_1_values:
        k_1=float(k_1)

        for k_2 in k_2_values:
            k_2=float(k_2)
            for rho in rho_values:
                rho=float(rho)
                for num_steps in num_steps_list:
                    num_steps=int(num_steps)

                    final_portfolio_prices_St=[]
                    simulatedprices_1_0=[]
                    avgsimulatedprices_1_0=[]
                    simulatedprices_1_1=[]
                    avgsimulatedprices_1_1=[]
                    avgfinal_portfolio_prices_St=[]
                    standard_deviation_of_the_simprices_1_0=[]
                    standard_deviation_of_the_simprices_1_1=[]

                    simulatedprices_2_0=[]
                    avgsimulatedprices_2_0=[]
                    simulatedprices_2_2=[]
                    avgsimulatedprices_2_2=[]
                    standard_deviation_of_the_simprices_2_0=[]
                    standard_deviation_of_the_simprices_2_2=[]

                    standard_deviation_of_the_final_portfolio_prices_St=[]
                    measure_values =[]
                    discounted_P_measure_values=[]
                    std_measure_values =[]

                    unhedged_values=[]
                    Avg_unhedged_values=[]

                    initial_portfolio_price_St=(alpha*S0_1+(1-alpha)*S0_2)
                    sigma=sqrt((alpha**2)*(sigma_1**2) +  ((1-alpha)**2)*(sigma_2**2)  +  2 * rho*(alpha)*(1-alpha) * sigma_1 * sigma_2)
                    black_scholes_callprice = black_scholes_calculator(initial_portfolio_price_St, K, r, T, sigma)

                    for run in range(num_simulations):
                        simulatedprice_1_0,simulatedprice_1_1,simulatedprice_2_0,simulatedprice_2_2,S_T,Measure,unhedged_value = portfolio_calculator(S0_1, S0_2, K, r, T, sigma_1, sigma_2, num_steps,N, mu_1, mu_2, rho, alpha)
                        simulatedprices_1_0.append(simulatedprice_1_0)
                        simulatedprices_1_1.append(simulatedprice_1_1)
                        simulatedprices_2_0.append(simulatedprice_2_0)
                        simulatedprices_2_2.append(simulatedprice_2_2)
                        final_portfolio_prices_St.append(S_T)
                        measure_values.append(Measure)
                        unhedged_values.append(unhedged_value)
                        
                    sum_price_1_0=sum(simulatedprices_1_0)
                    sum_price_1_1=sum(simulatedprices_1_1)
                    sum_finalstockprice=sum(final_portfolio_prices_St)

                    squared_diffs_1_0 = [(x - (sum_price_1_0/len(simulatedprices_1_0)) )** 2 for x in simulatedprices_1_0]
                    varianceofprices_1_0 = sum(squared_diffs_1_0) / len(simulatedprices_1_0)
                    std_dev_simprices_1_0 = math.sqrt(varianceofprices_1_0)
                    standard_deviation_of_the_simprices_1_0.append(std_dev_simprices_1_0)
                    avgsimulatedprice_1_0=np.sum(simulatedprices_1_0)/(len(simulatedprices_1_0))
                    avgsimulatedprices_1_0.append(avgsimulatedprice_1_0)

                    squared_diffs_1_1 = [(x - (sum_price_1_1/len(simulatedprices_1_1)) )** 2 for x in simulatedprices_1_1]
                    varianceofprices_1_1 = sum(squared_diffs_1_1) / len(simulatedprices_1_1)
                    std_dev_simprices_1_1 = math.sqrt(varianceofprices_1_1)
                    standard_deviation_of_the_simprices_1_1.append(std_dev_simprices_1_1)
                    avgsimulatedprice_1_1=np.sum(simulatedprices_1_1)/(len(simulatedprices_1_1))
                    avgsimulatedprices_1_1.append(avgsimulatedprice_1_1) 

                    sum_price_2_0=sum(simulatedprices_2_0)
                    squared_diffs_2_0 = [(x - (sum_price_2_0/len(simulatedprices_2_0)) )** 2 for x in simulatedprices_2_0]
                    varianceofprices_2_0 = sum(squared_diffs_2_0) / len(simulatedprices_2_0)
                    std_dev_simprices_2_0 = math.sqrt(varianceofprices_2_0)
                    standard_deviation_of_the_simprices_2_0.append(std_dev_simprices_2_0)
                    avgsimulatedprice_2_0=np.sum(simulatedprices_2_0)/(len(simulatedprices_2_0))
                    avgsimulatedprices_2_0.append(avgsimulatedprice_2_0)

                    sum_price_2_2=sum(simulatedprices_2_2)
                    squared_diffs_2_2 = [(x - (sum_price_2_2/len(simulatedprices_2_2)) )** 2 for x in simulatedprices_2_2]
                    varianceofprices_2_2 = sum(squared_diffs_2_2) / len(simulatedprices_2_2)
                    std_dev_simprices_2_2 = math.sqrt(varianceofprices_2_2)
                    standard_deviation_of_the_simprices_2_2.append(std_dev_simprices_2_2)
                    avgsimulatedprice_2_2=np.sum(simulatedprices_2_2)/(len(simulatedprices_2_2))
                    avgsimulatedprices_2_2.append(avgsimulatedprice_2_2)

                    squared_diff_finalstockprice = [(y - (sum_finalstockprice/len(final_portfolio_prices_St)) )** 2 for y in final_portfolio_prices_St]
                    varianceofprices_finalstockprice = sum(squared_diff_finalstockprice) / len(final_portfolio_prices_St)
                    std_dev_finalstockprice = math.sqrt(varianceofprices_finalstockprice)
                    standard_deviation_of_the_final_portfolio_prices_St.append(std_dev_finalstockprice)

                    avgS_T=np.sum(final_portfolio_prices_St)/(len(final_portfolio_prices_St))
                    avgfinal_portfolio_prices_St.append(avgS_T) 

                    discounted_P_measure_value=np.sum(measure_values)/(len(measure_values))
                    discounted_P_measure_values.append(discounted_P_measure_value)   

                    squared_diffs_measure = [(x - sum(measure_values)/(len(measure_values)) )** 2 for x in measure_values]
                    varianceofprices_measure = sum(squared_diffs_measure) / len(measure_values)
                    std_dev_measure = math.sqrt(varianceofprices_measure)
                    std_measure_values.append(std_dev_measure)
                    Avg_unhedged_value=np.sum(unhedged_values)/(len(unhedged_values))
                    Avg_unhedged_values.append(Avg_unhedged_value) 

                    hedging_the_right_and_wrong_asset_alpha_1_SECTION_3_2_2.append((alpha,rho,num_steps,k_1,k_2,black_scholes_callprice,avgsimulatedprice_1_1,std_dev_simprices_1_1,avgsimulatedprice_1_0,std_dev_simprices_1_0,avgsimulatedprice_2_2,std_dev_simprices_2_2,avgsimulatedprice_2_0,std_dev_simprices_2_0,avgS_T,std_dev_finalstockprice,discounted_P_measure_value,std_dev_measure,Avg_unhedged_value))

                    
df1 = pd.DataFrame(hedging_the_right_and_wrong_asset_alpha_1_SECTION_3_2_2, columns=['alpha','rho','num_steps','k_1','k_2','black_scholes_callprice','avgsimulatedprice_1_1', 'std_dev_simprices_1_1', 'avgsimulatedprice_1_0', 'std_dev_simprices_1_0','avgsimulatedprice_2_2','std_dev_simprices_2_2','avgsimulatedprice_2_0','std_dev_simprices_2_0','avgS_T','std_dev_finalstockprice','discounted_P_measure_value','std_dev_measure','Avg_unhedged_value'])

df1['alpha'] = df1['alpha'].round(2)
df1['rho'] = df1['rho'].round(2)
df1['num_steps'] = df1['num_steps'].round(2)
df1['k_2'] = df1['k_2'].round(2)
df1['k_1'] = df1['k_1'].round(2)

df1['black_scholes_callprice']=df1['black_scholes_callprice'].round(2)
df1['avgsimulatedprice_1_1'] = df1['avgsimulatedprice_1_1'].round(2)
df1['std_dev_simprices_1_1'] = df1['std_dev_simprices_1_1'].round(2)
df1['avgsimulatedprice_1_0'] = df1['avgsimulatedprice_1_0'].round(2)
df1['std_dev_simprices_1_0'] = df1['std_dev_simprices_1_0'].round(2)
df1['avgsimulatedprice_2_2'] = df1['avgsimulatedprice_2_2'].round(2)
df1['std_dev_simprices_2_2'] = df1['std_dev_simprices_2_2'].round(2)
df1['avgsimulatedprice_2_0'] = df1['avgsimulatedprice_2_0'].round(2)
df1['std_dev_simprices_2_0'] = df1['std_dev_simprices_2_0'].round(2)
df1['avgS_T'] = df1['avgS_T'].round(2)
df1['std_dev_finalstockprice'] = df1['std_dev_finalstockprice'].round(2)
df1['discounted_P_measure_value'] = df1['discounted_P_measure_value'].round(2)
df1['std_dev_measure'] = df1['std_dev_measure'].round(2)
df1['Avg_unhedged_value'] = df1['Avg_unhedged_value'].round(2)
df1.to_csv('hedging_the_right_and_wrong_asset_alpha_1_SECTION_3_2_2.csv', index=False, float_format='%.2f')

print("Results have been written to hedging_the_right_and_wrong_asset_alpha_1_SECTION_3_2_2.csv")


